In [3]:
#Import necessary libraries and modules
import random
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import f1_score
import time
from sklearn.model_selection import train_test_split

In [4]:
#Global Variables
img_data=[]
IMG_SIZE=50
X=[]
Y=[]

In [5]:
#Read Image
directories=os.listdir('C:/PROJECT/augmented data')
for folders in directories:
  for files in os.listdir('C:/PROJECT/augmented data/'+folders):
    img_temp=cv2.imread('C:/PROJECT/augmented data/'+folders+'/'+files,cv2.IMREAD_GRAYSCALE)
    img_temp=cv2.resize(img_temp,(IMG_SIZE,IMG_SIZE))
    img_data.append([img_temp,directories.index(folders)])

In [6]:
random.shuffle(img_data)

In [7]:
#Seperate X and Y
for x,y in img_data:
    X.append(x)
    Y.append(y)


In [8]:
#Reshape to 3D numpy
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)


In [9]:
#Save data inorder to eliminate the hassle of creating it again
np.save('X.npy',X)
np.save('Y.npy',Y)

In [10]:
#Seperate data to test, train and validate
def split_data(x, y, test_size=0.2):    
    x_train, x_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, test_size=0.5)
    
    return x_train, y_train, x_val, y_val, x_test, y_test

In [11]:
x_train,y_train,x_val,y_val,x_test,y_test=split_data(X,Y)

In [12]:
def plot_metrics(history):
    
    train_loss = history['loss']
    val_loss = history['val_loss']
    train_acc = history['acc']
    val_acc = history['val_acc']
    
    # Loss
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()
    
    # Accuracy
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [16]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(50,50,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=32, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(2, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [17]:
tuner_search=RandomSearch(build_model,
                          objective='val_acc',
                          max_trials=5,directory='output',project_name="BTC")

In [18]:
tuner_search.search(x_train,y_train,epochs=3,validation_split=0.1)

Train on 1486 samples, validate on 166 samples
Epoch 1/3
1486/1486 [==============================] - ETA: 36s - loss: 11.3184 - acc: 0.562 - ETA: 33s - loss: 157479.5030 - acc: 0.515 - ETA: 31s - loss: 104996.8020 - acc: 0.447 - ETA: 30s - loss: 86027.3710 - acc: 0.445 - ETA: 30s - loss: 69393.3169 - acc: 0.45 - ETA: 30s - loss: 57840.2005 - acc: 0.46 - ETA: 29s - loss: 49577.9366 - acc: 0.45 - ETA: 28s - loss: 43380.8454 - acc: 0.44 - ETA: 27s - loss: 38560.8486 - acc: 0.45 - ETA: 26s - loss: 34704.8417 - acc: 0.46 - ETA: 26s - loss: 31549.9212 - acc: 0.46 - ETA: 25s - loss: 28920.8182 - acc: 0.47 - ETA: 24s - loss: 26696.1915 - acc: 0.49 - ETA: 23s - loss: 24789.3709 - acc: 0.48 - ETA: 22s - loss: 23136.7914 - acc: 0.49 - ETA: 22s - loss: 21690.7863 - acc: 0.49 - ETA: 21s - loss: 20414.8989 - acc: 0.49 - ETA: 21s - loss: 19280.7768 - acc: 0.49 - ETA: 20s - loss: 18266.0354 - acc: 0.49 - ETA: 19s - loss: 17352.7699 - acc: 0.49 - ETA: 18s - loss: 16526.4796 - acc: 0.49 - ETA: 18s - lo

Train on 1486 samples, validate on 166 samples
Epoch 1/3
1486/1486 [==============================] - ETA: 23s - loss: 9.2184 - acc: 0.62 - ETA: 21s - loss: 211.5732 - acc: 0.56 - ETA: 21s - loss: 156.2264 - acc: 0.55 - ETA: 19s - loss: 123.6080 - acc: 0.57 - ETA: 18s - loss: 104.6080 - acc: 0.55 - ETA: 18s - loss: 88.2401 - acc: 0.5573 - ETA: 17s - loss: 75.7114 - acc: 0.571 - ETA: 16s - loss: 67.3882 - acc: 0.550 - ETA: 16s - loss: 60.2043 - acc: 0.531 - ETA: 15s - loss: 54.2697 - acc: 0.531 - ETA: 15s - loss: 49.4000 - acc: 0.525 - ETA: 14s - loss: 45.3402 - acc: 0.533 - ETA: 14s - loss: 41.9040 - acc: 0.540 - ETA: 13s - loss: 38.9548 - acc: 0.555 - ETA: 13s - loss: 36.4023 - acc: 0.558 - ETA: 12s - loss: 34.1635 - acc: 0.568 - ETA: 12s - loss: 32.1937 - acc: 0.569 - ETA: 12s - loss: 30.4392 - acc: 0.574 - ETA: 11s - loss: 28.8735 - acc: 0.577 - ETA: 11s - loss: 27.4610 - acc: 0.584 - ETA: 10s - loss: 26.1831 - acc: 0.590 - ETA: 10s - loss: 25.0228 - acc: 0.596 - ETA: 9s - loss: 23.

Train on 1486 samples, validate on 166 samples
Epoch 1/3
1486/1486 [==============================] - ETA: 52s - loss: 2.6524 - acc: 0.56 - ETA: 49s - loss: 206.6124 - acc: 0.50 - ETA: 47s - loss: 138.1707 - acc: 0.56 - ETA: 46s - loss: 109.2304 - acc: 0.56 - ETA: 45s - loss: 88.6808 - acc: 0.5813 - ETA: 44s - loss: 74.4817 - acc: 0.583 - ETA: 43s - loss: 63.9959 - acc: 0.575 - ETA: 42s - loss: 56.0825 - acc: 0.574 - ETA: 41s - loss: 49.9360 - acc: 0.566 - ETA: 41s - loss: 45.0142 - acc: 0.559 - ETA: 40s - loss: 40.9924 - acc: 0.556 - ETA: 39s - loss: 37.6334 - acc: 0.570 - ETA: 38s - loss: 34.7936 - acc: 0.572 - ETA: 37s - loss: 32.3550 - acc: 0.575 - ETA: 36s - loss: 30.2408 - acc: 0.583 - ETA: 34s - loss: 28.3998 - acc: 0.584 - ETA: 33s - loss: 26.7712 - acc: 0.584 - ETA: 32s - loss: 25.3198 - acc: 0.588 - ETA: 31s - loss: 24.0199 - acc: 0.592 - ETA: 30s - loss: 22.8594 - acc: 0.589 - ETA: 29s - loss: 21.8004 - acc: 0.590 - ETA: 27s - loss: 20.8400 - acc: 0.590 - ETA: 26s - loss: 19

Train on 1486 samples, validate on 166 samples
Epoch 1/3
1486/1486 [==============================] - ETA: 42s - loss: 4.0081 - acc: 0.46 - ETA: 38s - loss: 75.8292 - acc: 0.453 - ETA: 37s - loss: 92.2560 - acc: 0.479 - ETA: 36s - loss: 77.4315 - acc: 0.429 - ETA: 35s - loss: 67.6279 - acc: 0.456 - ETA: 35s - loss: 59.0412 - acc: 0.468 - ETA: 34s - loss: 51.0842 - acc: 0.473 - ETA: 33s - loss: 44.8505 - acc: 0.468 - ETA: 32s - loss: 39.9449 - acc: 0.482 - ETA: 31s - loss: 36.0187 - acc: 0.484 - ETA: 30s - loss: 32.8097 - acc: 0.494 - ETA: 29s - loss: 30.1260 - acc: 0.507 - ETA: 28s - loss: 27.8587 - acc: 0.507 - ETA: 28s - loss: 25.9116 - acc: 0.524 - ETA: 27s - loss: 24.2313 - acc: 0.525 - ETA: 26s - loss: 22.7563 - acc: 0.529 - ETA: 25s - loss: 21.4582 - acc: 0.536 - ETA: 24s - loss: 20.3032 - acc: 0.539 - ETA: 23s - loss: 19.2763 - acc: 0.532 - ETA: 22s - loss: 18.3444 - acc: 0.539 - ETA: 21s - loss: 17.5020 - acc: 0.540 - ETA: 20s - loss: 16.7339 - acc: 0.544 - ETA: 19s - loss: 16.

Train on 1486 samples, validate on 166 samples
Epoch 1/3
1486/1486 [==============================] - ETA: 34s - loss: 4.9129 - acc: 0.50 - ETA: 31s - loss: 55116.3432 - acc: 0.54 - ETA: 30s - loss: 36767.4352 - acc: 0.50 - ETA: 29s - loss: 27611.3036 - acc: 0.53 - ETA: 28s - loss: 22095.9808 - acc: 0.50 - ETA: 28s - loss: 18421.5563 - acc: 0.50 - ETA: 27s - loss: 15790.5125 - acc: 0.51 - ETA: 26s - loss: 13816.9652 - acc: 0.51 - ETA: 25s - loss: 12281.8805 - acc: 0.51 - ETA: 24s - loss: 11053.7781 - acc: 0.51 - ETA: 23s - loss: 10048.9553 - acc: 0.50 - ETA: 23s - loss: 9211.6119 - acc: 0.4948 - ETA: 22s - loss: 8503.0815 - acc: 0.490 - ETA: 22s - loss: 7895.7682 - acc: 0.493 - ETA: 21s - loss: 7369.4313 - acc: 0.477 - ETA: 20s - loss: 6908.8848 - acc: 0.486 - ETA: 20s - loss: 6502.5192 - acc: 0.494 - ETA: 19s - loss: 6141.3014 - acc: 0.498 - ETA: 18s - loss: 5818.6566 - acc: 0.501 - ETA: 18s - loss: 5527.7842 - acc: 0.492 - ETA: 17s - loss: 5264.5965 - acc: 0.494 - ETA: 16s - loss: 50

INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
log_file_name = f'brain_tumor_detection_cnn_{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs\{log_file_name}')
filepath="cnn-parameters-improvement-{epoch:02d}-{val_acc:.2f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

In [21]:
model.fit(x_train,y_train,batch_size=32, epochs=10, validation_data=(x_val,y_val),callbacks=[tensorboard, checkpoint])

Train on 1652 samples, validate on 207 samples
Epoch 1/10
1652/1652 [==============================] - ETA: 25s - loss: 0.7333 - acc: 0.62 - ETA: 22s - loss: 0.6252 - acc: 0.71 - ETA: 21s - loss: 0.5844 - acc: 0.72 - ETA: 20s - loss: 0.5693 - acc: 0.72 - ETA: 19s - loss: 0.5298 - acc: 0.75 - ETA: 19s - loss: 0.5356 - acc: 0.75 - ETA: 19s - loss: 0.5412 - acc: 0.74 - ETA: 19s - loss: 0.5289 - acc: 0.75 - ETA: 18s - loss: 0.5216 - acc: 0.75 - ETA: 18s - loss: 0.5151 - acc: 0.75 - ETA: 17s - loss: 0.5087 - acc: 0.76 - ETA: 17s - loss: 0.5122 - acc: 0.75 - ETA: 16s - loss: 0.5100 - acc: 0.74 - ETA: 16s - loss: 0.5063 - acc: 0.75 - ETA: 15s - loss: 0.5025 - acc: 0.75 - ETA: 15s - loss: 0.5088 - acc: 0.75 - ETA: 14s - loss: 0.5077 - acc: 0.75 - ETA: 14s - loss: 0.5060 - acc: 0.75 - ETA: 13s - loss: 0.5018 - acc: 0.75 - ETA: 13s - loss: 0.4964 - acc: 0.76 - ETA: 13s - loss: 0.4924 - acc: 0.76 - ETA: 12s - loss: 0.4971 - acc: 0.76 - ETA: 12s - loss: 0.4975 - acc: 0.76 - ETA: 11s - loss: 0.4907

In [36]:
#F1 Score Calculattion
y_pred=model.predict(x_test)

In [37]:
y_pred=np.where(y_pred>0.5,1,0)

In [38]:
y_pred=np.where(y_pred==1)

In [22]:
f1_score(y_test,y_pred[1])

0.9123543709
